In [2]:
import keras
import numpy as np
import os
import matplotlib.gridspec as gridspec
import tensorflow as tf
import matplotlib.pyplot as plt
#导入backend后端库
import keras.backend as K
#导入model类
from keras.models import Model
#需要的损失函数
from keras.losses import binary_crossentropy
#导入keras.layers中相关模块
from keras.layers import Dense,Input,Lambda
#导入优化器
from keras.optimizers import Adam
#导入callback回调函数
import keras.callbacks as callb
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
if not os.path.exists('out/'):
    os.makedirs('out/')

#画图方法
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

#定义参数
batch_size = 300
epochs = 100
step = 20
input_dim = 784
z_dim = 2
input_shape = (input_dim,)
z_shape = (z_dim,)


#读取数据
x_train = mnist.train.images
x_test = mnist.test.images
x_train /= 255.
x_test /= 255.


input_img = Input(input_shape)

encode = Dense(512,activation='relu',
                kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.001))(input_img)
encode = Dense(256,activation='relu',
               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.001))(encode)

mu = Dense(z_dim)(encode)
logvar = Dense(z_dim)(encode)


#自定义函数
def get_z(args):
    z_mu, z_logvar = args
    epsilon = K.random_normal(shape=K.shape(z_mu))
    return z_mu + K.exp(z_logvar / 2) * epsilon


z = Lambda(get_z)([mu,logvar])

decode_1 = Dense(256,activation='relu',
               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.001))
decode_2 = Dense(512,activation='relu',
               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.001))
decode_3 = Dense(input_dim,activation='sigmoid',
               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.001))
decode = decode_1(z)
decode = decode_2(decode)
logits = decode_3(decode)


#修改优化器中的学习率
adam = Adam(lr=0.001,decay=0.0005)
#创建模型
vae = Model(input_img,logits)


def myloss(y_true,y_pred):
    BCE = binary_crossentropy(y_true,y_pred)
    #KL散度
    KLD = 0.5*K.sum(K.pow(mu, 2) + K.exp(logvar) - 1 - logvar,1)
    # VAE loss
    return K.mean(KLD + BCE)


#编译模型
vae.compile(optimizer=adam,loss=myloss)

#定义每轮结束操作
def epoch_end_operation(epoch):
    if epoch%step == 0:
        loss = vae.evaluate(x_test,x_test,verbose=0)
        print('第%s轮测试后结果：' % epoch)
        print('loss:%.4f' % loss)
        print()
        i = 0
        i += int(epoch/step)
        test_z = np.random.randn(16, z_dim)
        input_z = Input(z_shape)
        test_decode = decode_1(input_z)
        test_decode = decode_2(test_decode)
        test_logits = decode_3(test_decode)
        generate = Model(input_z,test_logits)
        pred_samples = generate.predict(test_z)
        fig = plot(pred_samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')

#自定义回调函数
class Pred_Plot(callb.Callback):
    def on_epoch_end(self, epoch, logs={}):
        epoch_end_operation(epoch+1)
        
pred_plot = Pred_Plot()

#需要执行的回调函数列表
callback_list = [pred_plot]

vae.fit(x_train,x_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        callbacks=callback_list)

第20轮测试后结果：
loss:0.0040

第40轮测试后结果：
loss:0.0040

第60轮测试后结果：
loss:0.0040

第80轮测试后结果：
loss:0.0040

第100轮测试后结果：
loss:0.0040

